# Calculating Selection Function values for combined surveys

In [1]:
# autoreload modules when files have been altered
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os

import numpy as np, pandas as pd

sys.path.append('../seestar/')
import Combine, SelectionGrid, IsochroneScaling

In [20]:
folder = '.../Galaxia_data'

# Import Selection function

In [11]:
survey = pd.read_csv(folder + '/Galaxia_sf_survey.csv')
survey['Colour'] = survey.Japp-survey.Kapp

pointings = pd.read_csv(folder + '/Galaxia_sf_fieldinfo.csv')
pointings = pointings[['fieldID', 'glon', 'glat', 'halfangle','Magmin', 'Magmax', 'Colmin', 'Colmax']]
pointings.rename({'glon':'phi', 'glat':'theta'})

def get_spectro(field):
    
    field_data = survey[survey.fieldID==field]
    field_data = np.array(field_data[['Happ', 'Colour']])
    
    return field_data

def get_photo(field):
    
    file_path = folder + '/photometric/'+str(field)+'.csv'
    field_data = pd.read_csv(file_path, usecols=['Japp', 'Happ', 'Kapp'])
    
    field_data['Colour'] = field_data.Japp - field_data.Kapp
    field_data = np.array(field_data[['Happ', 'Colour']])
    
    return field_data

## I'm actually going to load the same selection function twice
- This simulates running exactly the same survey twice then combining the results

In [13]:
SF1 = SelectionGrid.SFGenerator(get_spectro, get_photo, pointings, ncores=3,
                              spectro_model=('GMM', 3), photo_model=('GMM', 3))
SF1.load_obsSF(folder+'/Galaxia_obsSF.pickle')

The spectro model description is:('GMM', 3)
The photo model description is:('GMM', 3)
Unpickling colour-magnitude interpolant dictionaries...
...done.



In [14]:
SF2 = SelectionGrid.SFGenerator(get_spectro, get_photo, pointings, ncores=3,
                              spectro_model=('GMM', 3), photo_model=('GMM', 3))
SF2.load_obsSF(folder+'/Galaxia_obsSF.pickle')

The spectro model description is:('GMM', 3)
The photo model description is:('GMM', 3)
Unpickling colour-magnitude interpolant dictionaries...
...done.



## Load in a dataset

In [4]:
%%time
# Load in the isochrone interpolant
IsoCalculator = IsochroneScaling.IntrinsicToObservable()
IsoCalculator.LoadColMag(folder+'/isochrones/isochrone_interpolantinstances.pickle')

CPU times: user 5.68 s, sys: 1.26 s, total: 6.95 s
Wall time: 6.98 s


In [5]:
# Load in field 2, 3 photometric data
full = pd.DataFrame()
fields= [1.0,2.0,3.0]
for field in fields:
    full = pd.concat((full, pd.read_csv(folder+'/photometric/'+str(field)+'.csv')))
full['Colour'] = full['Japp']-full['Kapp']
full.rename(index=str, columns={'feh':'mh', 'smass':'mass','rad':'s'}, inplace=True)
# Recalculate to make colour and apparent magnitude agree
full['Colour'], full['Happ'] = IsoCalculator.ColourMapp(full.ageGyr, full.mh, full.mass, full.s)

## Calculate SF prob for each Selection Function

In [15]:
sf_prob1 = \
    SF1(full, method='observable', coords=['Happ', 'Colour'], angle_coords=['glon', 'glat']).union

Calculating all SF values...
...Assigning: 2176244/2176244        
Calculating: 2176244/2176244        ...done
Calculating union contribution...
...done


In [16]:
%%time
sf_prob2 = \
    SF2(full, method='observable', coords=['Happ', 'Colour'], angle_coords=['glon', 'glat']).union

Calculating all SF values...
...Assigning: 2176244/2176244        
Calculating: 2176244/2176244        ...done
Calculating union contribution...
...done
CPU times: user 1min 23s, sys: 2.2 s, total: 1min 25s
Wall time: 1min 24s


In [17]:
%%time
sf_prob2 = \
    SF2(full, method='observable', coords=['Happ', 'Colour'], angle_coords=['glon', 'glat']).union

Calculating all SF values...
...Assigning: 2176244/2176244        
Calculating: 2176244/2176244        ...done
Calculating union contribution...
...done
CPU times: user 2min, sys: 2.2 s, total: 2min 2s
Wall time: 2min 2s


## Calculate Combined selection function

In [17]:
sf_combined = Combine.union_iterate(np.stack((sf_prob1, sf_prob2)).T)

In [18]:
full['sf_values'] = sf_combined

In [19]:
full

,fieldID,ageGyr,mh,mass,s,glat,glon,Japp,Kapp,Happ,Colour,sf_values
0,0,0.020185,-0.050110,0.476705,0.048435,-0.011622,0.554240,9.168076,8.374977,8.505083,0.793098,0.038704
1,0,0.049355,-0.064495,0.129319,0.048628,-0.015517,0.544759,11.740240,10.959921,11.189833,0.780319,0.013194
2,0,0.037470,-0.150292,0.213953,0.031611,-0.017182,0.529824,9.801041,9.066318,9.250110,0.734723,0.034238
3,0,0.008633,0.100070,0.378792,0.085587,0.008152,0.500311,10.066890,9.317022,9.495631,0.749867,0.031580
4,0,0.141066,0.114381,0.155861,0.081688,0.012797,0.524431,13.286500,12.513443,12.741357,0.773058,0.025356
5,0,0.050807,-0.079869,0.852139,0.092737,0.032549,0.532585,9.311171,8.855659,8.909633,0.455512,0.000000
6,0,0.136857,-0.015897,0.503747,0.091405,-0.033502,0.531574,11.247476,10.476863,10.601937,0.770613,0.018547
7,0,0.003993,0.210433,0.178984,0.090270,-0.024329,0.548396,10.650079,9.888104,10.126838,0.761975,0.024131
8,0,0.060373,0.045632,0.574671,0.101494,-0.025786,0.507670,10.966142,10.179445,10.296063,0.786696,0.021762
9,0,0.080834,-0.212656,0.108878,0.089210,0.021749,0.527155,13.707810,12.923596,13.155641,0.784214,0.014874
